In [1]:
from math import sqrt

#from bertviz import head_view
#from bertviz.transformers_neuron_view import BertModel
#from bertviz.neuron_view import show
import torch
from torch import nn
import torch.nn.functional as F
from transformers import AutoConfig, AutoModel, AutoTokenizer

/Users/damiansp/miniforge3/lib/python3.9/site-packages/jax/_src/lib/__init__.py:32: UserWarning: JAX on Mac ARM machines is experimental and minimally tested. Please see https://github.com/google/jax/issues/5501 in the event of problems.
  warnings.warn("JAX on Mac ARM machines is experimental and minimally tested. "


In [2]:
model_ckpt = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
#model = BertModel.from_pretrained(model_ckpt)

In [3]:
text = 'time flies like an arrow'
#show(
#    model, 
#    'bert',
#    tokenizer,
#    text,
#    display_mode='light',
#    layer=0,
#    head=8)

In [4]:
inputs = tokenizer(text, return_tensors='pt', add_special_tokens=False)
inputs.input_ids

tensor([[ 2051, 10029,  2066,  2019,  8612]])

In [5]:
config = AutoConfig.from_pretrained(model_ckpt)
token_emb = nn.Embedding(config.vocab_size, config.hidden_size)
token_emb

Embedding(30522, 768)

In [6]:
input_embeds = token_emb(inputs.input_ids)
input_embeds.size() # one batch of our 5 words in a 768-vector encoding

torch.Size([1, 5, 768])

In [7]:
query = key = value = input_embeds
dim_k = key.size(-1)
scores = torch.bmm(query, key.transpose(1, 2)) / sqrt(dim_k)
scores.size()

torch.Size([1, 5, 5])

In [8]:
scores

tensor([[[28.2861, -0.8173, -0.5413, -1.0572,  1.7418],
         [-0.8173, 30.6034, -0.8678, -1.2241, -1.9627],
         [-0.5413, -0.8678, 26.6746, -0.8248,  0.7390],
         [-1.0572, -1.2241, -0.8248, 24.8298, -0.4212],
         [ 1.7418, -1.9627,  0.7390, -0.4212, 27.5537]]],
       grad_fn=<DivBackward0>)

In [9]:
weights = F.softmax(scores, dim=-1)
weights

tensor([[[1.0000e+00, 2.2939e-13, 3.0230e-13, 1.8047e-13, 2.9647e-12],
         [2.2602e-14, 1.0000e+00, 2.1488e-14, 1.5048e-14, 7.1896e-15],
         [1.5145e-12, 1.0926e-12, 1.0000e+00, 1.1407e-12, 5.4490e-12],
         [5.7205e-12, 4.8410e-12, 7.2170e-12, 1.0000e+00, 1.0805e-11],
         [6.1668e-12, 1.5177e-13, 2.2623e-12, 7.0899e-13, 1.0000e+00]]],
       grad_fn=<SoftmaxBackward0>)

In [10]:
weights.sum(dim=-1)

tensor([[1., 1., 1., 1., 1.]], grad_fn=<SumBackward1>)

In [11]:
attn_outputs = torch.bmm(weights, value)
attn_outputs.shape

torch.Size([1, 5, 768])

In [12]:
# wrap it up
def scaled_dot_product_attention(query, key, value):
    dim_k = key.size(-1)
    scores = torch.bmm(query, key.transpose(1, 2)) / sqrt(dim_k)
    weights = F.softmax(scores, dim=-1)
    attn = torch.bmm(weights, value)
    return attn

In [13]:
class AttentionHead(nn.Module):
    def __init__(self, embed_dim, head_dim):
        super().__init__()
        self.q = nn.Linear(embed_dim, head_dim)
        self.k = nn.Linear(embed_dim, head_dim)
        self.v = nn.Linear(embed_dim, head_dim)
        
    def forward(self, hidden_state):
        attn_outputs = scaled_dot_product_attention(
            self.q(hidden_state),
            self.k(hidden_state),
            self.v(hidden_state))
        return attn_outputs

In [14]:
class MultiHeadAttention(nn.Module):
    def __init__(self, config):
        super().__init__()
        embed_dim = config.hidden_size
        n_heads = config.num_attention_heads
        head_dim = embed_dim // n_heads
        self.heads = nn.ModuleList(
            [AttentionHead(embed_dim, head_dim) for _ in range(n_heads)]
        )
        self.output_linear = nn.Linear(embed_dim, embed_dim)
        
    def forward(self, hidden_state):
        x = torch.cat([head(hidden_state) for head in self.heads],
                      dim=-1)
        x = self.output_linear(x)
        return x

In [15]:
multihead_attn = MultiHeadAttention(config)
attn_output = multihead_attn(input_embeds)
attn_output.size() # batch, words, embedding vec

torch.Size([1, 5, 768])

In [16]:
mod = AutoModel.from_pretrained(model_ckpt, output_attentions=True)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [17]:
sent_a = 'time flies like an arrow'
sent_b = 'fruit flies like a banana'

viz_inputs = tokenizer(sent_a, sent_b, return_tensors='pt')
attention = mod(**viz_inputs).attentions
sent_b_start = (viz_inputs.token_type_ids == 0).sum(dim=1)
tokens = tokenizer.convert_ids_to_tokens(viz_inputs.input_ids[0])

In [18]:
#head_view(attention, tokens, sent_b_start, heads=[8])

In [19]:
class FeedForward(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.lin1 = nn.Linear(config.hidden_size,
                              config.intermediate_size)
        self.lin2 = nn.Linear(config.intermediate_size,
                              config.hidden_size)
        self.gelu = nn.GELU()
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        
    def forward(self, x):
        x = self.lin1(x)
        x = self.gelu(x)
        x = self.lin2(x)
        x = self.dropout(x)
        return x

In [20]:
ff = FeedForward(config)
ff_outputs = ff(attn_outputs)
ff_outputs.size()

torch.Size([1, 5, 768])

In [21]:
class TransformerEncoderLayer(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.layer_norm1 = nn.LayerNorm(config.hidden_size)
        self.layer_norm2 = nn.LayerNorm(config.hidden_size)
        self.attention = MultiHeadAttention(config)
        self.feed_forward = FeedForward(config)
        
    def forward(self, x):
        # Apply layer norm, then copy into q, k, v
        hidden_state = self.layer_norm1(x)
        # Attention with skip
        x = x + self.attention(hidden_state)
        # ...and FF with skip
        x = x + self.feed_forward(self.layer_norm2(x))
        return x

In [22]:
encoder_layer = TransformerEncoderLayer(config)
input_embeds.shape, encoder_layer(input_embeds).size()

(torch.Size([1, 5, 768]), torch.Size([1, 5, 768]))

In [31]:
# incorporate positional encodings
class Embeddings(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.token_embeddings = nn.Embedding(config.vocab_size,
                                             config.hidden_size)
        self.positional_embeddings = nn.Embedding(
            config.max_position_embeddings, config.hidden_size)
        self.layer_norm = nn.LayerNorm(config.hidden_size, eps=1e-12)
        self.dropout = nn.Dropout()
        
    def forward(self, input_ids):
        seq_len = input_ids.size(1)
        position_ids = (torch
                        .arange(seq_len, dtype=torch.long)
                        .unsqueeze(0))
        token_embeddings = self.token_embeddings(input_ids)
        positional_embeddings = self.positional_embeddings(position_ids)
        embeddings = token_embeddings + positional_embeddings
        embeddings = self.layer_norm(embeddings)
        embeddings = self.dropout(embeddings)
        return embeddings

In [32]:
embedding_layer = Embeddings(config)
embedding_layer(inputs.input_ids).size()

torch.Size([1, 5, 768])

In [33]:
class TransformerEncoder(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.embeddings = Embeddings(config)
        self.layers = nn.ModuleList(
            [TransformerEncoderLayer(config) 
             for _ in range(config.num_hidden_layers)])
        
    def forward(self, x):
        x = self.embeddings(x)
        for layer in self.layers:
            x = layer(x)
        return x

In [34]:
encoder = TransformerEncoder(config)
encoder(inputs.input_ids).size()

torch.Size([1, 5, 768])